**AI & Machine Learning (KAN-CINTO4003U) - Copenhagen Business School | Spring 2025**

***


# Part I: Bag-of-Words Model

Please see the description of the assignment in the README file (section 1) <br>
**Guide notebook**: [guides/bow_guide.ipynb](guides/bow_guide.ipynb)


***

<br>

* Note that you should report results using a classification report. 

* Also, remember to include some reflections on your results: Are there any hyperparameters that are particularly important?

* You should follow the steps given in the `bow_guide` notebook

<br>

***

In [44]:
# imports for the project

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

### 1. Load the data

We can load this data directly from [Hugging Face Datasets](https://huggingface.co/docs/datasets/) - The HuggingFace Hub- into a Pandas DataFrame. Pretty neat!

**Note**: This cell will download the dataset and keep it in memory. If you run this cell multiple times, it will download the dataset multiple times.

You are welcome to increase the `frac` parameter to load more data.

In [45]:

splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}

train = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["train"])
test = pd.read_parquet("hf://datasets/fancyzhx/ag_news/" + splits["test"])

print(train.shape, test.shape)

(120000, 2) (7600, 2)


In [46]:

label_map = {
    0: 'World',
    1: 'Sports',
    2: 'Business',
    3: 'Sci/Tech'
}

def preprocess(df: pd.DataFrame, frac : float = 1e-2, label_map : dict[int, str] = label_map, seed : int = 42) -> pd.DataFrame:
    """ Preprocess the dataset 

    Operations:
    - Map the label to the corresponding category
    - Filter out the labels not in the label_map
    - Sample a fraction of the dataset (stratified by label)

    Args:
    - df (pd.DataFrame): The dataset to preprocess
    - frac (float): The fraction of the dataset to sample in each category
    - label_map (dict): A mapping of the original label to the new label
    - seed (int): The random seed for reproducibility

    Returns:
    - pd.DataFrame: The preprocessed dataset
    """

    return  (
        df
        .assign(label=lambda x: x['label'].map(label_map))
        [lambda df: df['label'].isin(label_map.values())]
        .groupby('label')[["text", "label"]]
        .apply(lambda x: x.sample(frac=frac, random_state=seed))
        .reset_index(drop=True)

    )

train_df = preprocess(train, frac=0.01)
test_df = preprocess(test, frac=0.1)

# clear up some memory by deleting the original dataframes
del train
del test

train_df.shape, test_df.shape

((1200, 2), (760, 2))

### 2. Split the data
The AG News dataset is already split into training and test sets. 
We will use Scikit-Learn's `train_test_split` to split the training set into a training and validation set. 
We will use the training set for training our BoW model and the validation set for evaluating the model

In [56]:
(
    
    X_train,
    X_val,
    y_train,
    y_val

) = train_test_split(train_df["text"], train_df["label"], test_size=0.2, random_state=42)

print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(960,) (240,) (960,) (240,)


### 3. Build the BoW model

We will use the `fit_transform` method to build the BoW model. We will also use the `transform` method to transform the validation set into a BoW representation.
We will __not__ fit our `CountVectorizer` on the validation set. We only fit it on the training set. This is important to avoid data leakage.


In [57]:
# countvectorizer
cv = CountVectorizer(stop_words='english') # This removes common words like "the", "is", "and", reducing noise in the model.
X_train_vectorized = cv.fit_transform(X_train)

In [58]:
print(X_train_vectorized.shape)


(960, 7395)


We have 960 documents (rows) and 7634 unique words in the training set (columns)


In [59]:
X_train_vectorized.todense()


matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

`X_train_vectorized.todense()` converts a sparse matrix into a dense matrix, explicitly storing all values (including zeros). This makes it easier to inspect but consumes more memory.

### 4. Create a classifier

Below we create a simple classifier using Scikit-Learn's [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). We train the classifier on the BoW representation of the training set and evaluate it on the BoW representation of the validation set.

In [69]:
lr_clf = LogisticRegression(max_iter=500, C=0.5, solver="liblinear")  # Increase iterations, adjust regularization


lr_clf.fit(X_train_vectorized, y_train)

LogisticRegression(C=0.5, max_iter=500, solver='liblinear')

### Parameters:
max_iter=500: Ensures convergence.

C=1.0: Controls regularization strength.

solver="liblinear": Suitable for small datasets.


### 5. Get predictions

In [70]:
X_val_vectorized = cv.transform(X_val) # note that we use transform here, not fit_transform

y_pred = lr_clf.predict(X_val_vectorized)

### 6. Evaluate BoW model

In [71]:

print("Performance on the training set:")
print(classification_report(y_train, lr_clf.predict(X_train_vectorized), target_names=label_map.values()))

print("Performance on the validation set:")
print(classification_report(y_val, y_pred, target_names=label_map.values()))

Performance on the training set:
              precision    recall  f1-score   support

       World       1.00      1.00      1.00       238
      Sports       1.00      1.00      1.00       240
    Business       1.00      1.00      1.00       240
    Sci/Tech       1.00      1.00      1.00       242

    accuracy                           1.00       960
   macro avg       1.00      1.00      1.00       960
weighted avg       1.00      1.00      1.00       960

Performance on the validation set:
              precision    recall  f1-score   support

       World       0.78      0.69      0.74        62
      Sports       0.80      0.65      0.72        60
    Business       0.81      0.90      0.85        60
    Sci/Tech       0.74      0.88      0.80        58

    accuracy                           0.78       240
   macro avg       0.78      0.78      0.78       240
weighted avg       0.78      0.78      0.78       240



In [72]:
test_df_vectorized = cv.transform(test_df["text"])

print("Performance on the test set:")
print(classification_report(test_df["label"], lr_clf.predict(test_df_vectorized), target_names=label_map.values()))

Performance on the test set:
              precision    recall  f1-score   support

       World       0.78      0.78      0.78       190
      Sports       0.84      0.77      0.80       190
    Business       0.88      0.91      0.90       190
    Sci/Tech       0.83      0.87      0.85       190

    accuracy                           0.83       760
   macro avg       0.83      0.83      0.83       760
weighted avg       0.83      0.83      0.83       760



### Overfitting on the Training Set
Training accuracy: 100%
Validation accuracy: 78%
Test accuracy: 83%

The model memorized the training data completely but doesn't generalize as well to unseen data. This suggests:
 - The model is too complex or lacks regularization.
 - The dataset is small, so the model overfits easily.
- Bag-of-Words may capture spurious patterns instead of general linguistic features.

Validation accuracy (78%) is lower than test accuracy (83%)—this suggests that either:

- The validation set is too small and not fully representative.
- The test set happens to be easier (less noise, clearer separation).